In this notebook, we are going to implement the method of calculating information gain based on entropy.
This is an educational notebook and I request the readers to report any bugs to <b>AbdhMogammady@gmail.com</b> or comment on my github.
I tried to write this notebook so that it can be used for other datasets, currently this notebook can be used for datasets with categorical columns and two-class for target column.
At the end of the notebook, after obtaining the highest information gain,you can use the column found as the root of the decision tree algoritm.

The entropy of the original dataset is calculated based on the target column named 'Buy'.
Suppose our dataset is D and has k = 2 is number of classes in the target column,we index them as class_i and the total number of records in the dataset is equal to N, so:

n(D)=N ==> P(class_i) = n(class_i)/n(D) = p_i

and Entropy of D is:

Entropy(D) = -( p_1*Ln(p_1)+ P_2*Ln(p_2)+...+P_k*Ln(p_k) )

This article will be updated.

In [178]:
import pandas # to working with dataset
import math   # to working with math functions like log10

In [179]:
#This is a very sample dataset to learn Gain information rules
df = pandas.read_csv('Gain-Information-sample.csv')

# number of record we read by pandas from our dataset, 
# we use this value as size of the sample to claculte probability of each class
n_records = len(df)

# display all data
df.head()


,Credit,Age,Income,Education,Buy
0,Good,Young,High,Good,No
1,Excellent,Young,High,Good,No
2,Good,Middle,High,Good,Yes
3,Good,Old,Middle,Good,Yes
4,Good,Old,High,Bad,Yes


In [180]:
group_object = df.groupby(by="Buy")

buy_groups_dict = group_object.groups

print("Buy groups dictionary:\n",buy_groups_dict)

group_object.describe()


Buy groups dictionary:
 {'No': [0, 1, 5, 7, 13], 'Yes': [2, 3, 4, 6, 8, 9, 10, 11, 12]}


Credit                          Age                     Income         \
     count unique        top freq count unique     top freq  count unique   
Buy                                                                         
No       5      2  Excellent    3     5      2   Young    3      5      2   
Yes      9      2       Good    6     9      3  Middle    4      9      2   

               Education                    
      top freq     count unique   top freq  
Buy                                         
No   High    3         5      2  Good    4  
Yes  High    5         9      2   Bad    6

In [185]:
#Compute entropy of the dataset

#Calculate the frequency of each item in the group

dataset_entropy = 0

for key in buy_groups_dict:
    #list of real index of items in the dataset
    index_list = buy_groups_dict.get(str(key)).to_list()
    n_class_item = len(index_list)
    prob_class_item = n_class_item/n_records
    #print("Class '"+ key+"',frequency =" ,n_class_item,',P('+key+") = ",prob_class_item)
    
    #print()
    
    log_class_item = math.log(prob_class_item)
    
    #print("P("+key+") x Log(p("+key+"))=",prob_class_item*log_class_item)
    
    #print()
    
    dataset_entropy = dataset_entropy + prob_class_item*log_class_item
    
dataset_entropy = - dataset_entropy

print("Dataset Entropy : ",dataset_entropy)
    

Dataset Entropy :  0.6517565611726531


Computing entropy of a column

In [182]:
# get list of column name
columns = list(df.columns.values)

# no need target column to calculate entropy and gain information
del columns[4] 

columns

['Credit', 'Age', 'Income', 'Education']

In [210]:

# initalize empty dictionary to store 'Gain Information' of each column 
gain_info_dict = dict()

column_entropy = 0

for column_ in columns:
    
    column_entropy = 0
    group_object = df.groupby(by=column_)

    classes = list(group_object.groups.keys())

    for class_ in classes:

        n_yes = 0
        n_no  = 0
    
        for i in range(n_records):
            if df[column_][i]==class_:
                if df['Buy'][i]=='Yes': n_yes+=1
                if df['Buy'][i]=='No' : n_no+=1

        #Number of subclass items
        n_class = n_yes + n_no

        p_yes = 0
        
        if n_yes != 0 and n_class != 0 :p_yes = n_yes/n_class
            
        p_no  = 0
        
        if n_no != 0 and n_class != 0 :p_no = n_no/n_class
        
        class_entropy = 0
        
        if p_yes != 0 : class_entropy = p_yes*math.log(p_yes)
            
        if p_no != 0 : class_entropy += p_no*math.log(p_no)
        
        class_entropy = - class_entropy

        column_entropy += (n_class/n_records)* class_entropy
    
        #print("\nEntropy of the class of '"+class_+"' : ",class_entropy)
    gain_info_dict[column_] = dataset_entropy - column_entropy
    
    #print("\nEntropy of column '"+column_+"' : ",column_entropy) 
    print("\nGain information of column '"+column_+"' : ",dataset_entropy - column_entropy) 
    


Gain information of column 'Credit' :  0.033359115436214726

Gain information of column 'Age' :  0.17103394188032706

Gain information of column 'Income' :  0.0009286386703150074

Gain information of column 'Education' :  0.10524434967821283


In [211]:
#find item with maximom value in the dictionary
max_item = list(gain_info_dict.keys())[0]

for item in gain_info_dict:
    if gain_info_dict[item]> gain_info_dict[max_item]:
        max_item = item

print("The column with the highest information gain:\n\n\tColumn '"
      + max_item+"'\n\tIngormation Gain : ",gain_info_dict[max_item])

The column with the highest information gain:

	Column 'Age'
	Ingormation Gain :  0.17103394188032706


<b>Now the column found can be used for the root of the decision tree algorithm</b>